## Tripadvisor評論情緒分析與飯店星級預測 —— 報告講稿（口頭用）

### 一、專案背景與動機

各位好，我們這組分析的主題是「Tripadvisor 飯店星級預測與評論主題分析」。

在現代旅遊中，消費者常常依賴網路評論選擇飯店，因此，我們想探討：

用戶評論內容與飯店星級之間是否具有關聯性？

能否利用文字評論預測飯店的星級？

這樣的模型將能：

協助平台預測未標星飯店星級。

提供業者改進方向。

幫助消費者更透明做出選擇。

========================================================================
### 二、資料介紹與前處理

我們使用兩份資料：

offerings.csv：飯店資訊（包含星等、地址）

reviews.csv：評論內容

共計超過 84 萬筆評論，平均每家飯店約有 200 筆以上。

處理方式包括：

文字清洗（移除符號、數字、小寫化）

合併資料集，並刪除空值

========================================================================
### 三、TF-IDF + 多模型訓練與交叉驗證

接著，我們使用 TF-IDF 將評論文字向量化，設定 max_features=500。

並以 70%/30% 的比例切分資料，訓練三種模型：

Logistic Regression

Decision Tree

Random Forest

使用五折交叉驗證，評估 weighted avg F1-score。

🔍 Logistic Regression 表現最佳，F1-score 為 0.38

🟡 模擬問答：交叉驗證與指標選擇

🎤 Q1：你們使用五折交叉驗證，為什麼評估指標選擇的是 weighted avg F1-score？

🎙 我們選擇 weighted avg F1-score 是因為：

星級分布不平均，像 3 星評論非常多，1 星或 5 星則偏少。

weighted avg 會依照每類別樣本數加權計算 F1-score，更真實反映整體模型表現。

相較之下，macro avg 會高估小樣本類別，accuracy 又容易受到大類別偏誤影響。

========================================================================
### 四、模型可解釋性：關鍵詞視覺化

使用 logistic regression 的係數 coef_ 找出對預測各星級影響最大的詞：

⭐ 高星級關鍵詞：excellent, amazing, friendly, perfect

⭐ 低星級關鍵詞：dirty, terrible, worst, bugs

⭐ 中星級詞彙則偏向模稜兩可：ok, average, decent

這部分也幫助我們更理解模型決策依據。

🎤 Q2：為什麼不用 accuracy 當評估指標？

🎙 因為 accuracy 無法反映類別不平衡問題。
舉例：若大多評論是 3 星，只要模型都猜 3 星就會很高準確率，
但實際上根本沒預測出 1 星或 5 星。
➡️ F1-score 綜合 precision 和 recall，更能衡量模型實用性。

========================================================================
### 五、LDA 主題模型分析

我們再進一步將評論分為 4 個星等區間（12、23、34、45），
對每組使用 gensim 的 LDA 主題模型。

每組預設提取 5 個主題，並使用 pyLDAvis 進行互動式可視化。

🔍 我們觀察到：

某些主題明顯為不同語言（如西班牙文、德文）

也有些主題雜揉正負情緒詞，模型會將共同出現視為同一主題

🎤 Q3：主題模型中為什麼會出現非英文詞如 'de', 'le'？

🎙 Tripadvisor 是國際平台，很多評論來自歐洲與拉美旅客，原始語言就不是英文。
這些詞進入斷詞流程後，沒有被過濾，就進了 LDA 模型。
後續我們會考慮用語言識別工具來排除或分流非英文資料。

========================================================================
### 六、主題可視化與整體分布解釋

透過 corpus2csc 將 LDA 分布矩陣化，畫出整體主題在評論中的平均分佈：

例如：在 1~2 星評論中，

Topic 4（clean, place, staff）佔比最高（約 40%）

Topic 1（義大利文 'e', 'per'）佔比只有 4%

🔍 我們發現主題詞有時與語意極性無關，因此下一步導入情緒分析來強化解釋力。

🎤 Q4：pyLDAvis 裡的 PC1 與 PC2 是什麼意思？

🎙 它是用 PCA 或 MDS 將高維主題分佈降到 2D 平面：

PC1, PC2 是投影出來的主成分，代表主題之間語意變異最大的方向。

圓圈越分開，代表主題差異越大；越大代表出現比例高。

========================================================================
### 七、結論與後續方向

我們可以有效地從評論中預測星級，Logistic Regression 表現最佳。

詞彙與星級間具明顯情緒傾向關聯，可用於可視化與推薦。

LDA 模型能發掘主題，但需進一步結合情緒詞典或 TextBlob、VADER 分析提升可讀性。